<a href="https://colab.research.google.com/github/AnalyticsByte/Case-Studies/blob/master/Twitter_CNN_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Dependencies**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup


In [ ]:
try:
   %tensorflow_version 2x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import layers

import tensorflow_datasets as tfds

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2x`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


# **Data Preprocessing**

In [ ]:
cols = ["sentiment", "id", "date", "query", "user", "text"]

train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Dataset/CNN_NLP/training.csv',
                          header=None, names=cols, engine="python", encoding="latin1")

test_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Dataset/CNN_NLP/testdata.csv',
                          header=None, names=cols, engine="python", encoding="latin1")

In [ ]:
train_data.head(3)

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...


### **Cleaning**

In [ ]:
train_data.drop(["id","date","query", "user"], 
          axis = 1,
          inplace=True)

In [ ]:
train_data.head(5)

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
def clean_tweet(tweet):
  tweet = BeautifulSoup(tweet, "lxml").get_text()
  tweet = re.sub(r"@[A-Za-z0-9]+", '', tweet) #regular exp (re) https://regexr.com/
  tweet = re.sub(r"https?://[A-Za-z0-9./]+", '', tweet)
  tweet = re.sub(r"[^a-zA-Z.!?']", '', tweet)
  tweet = re.sub(r" +", '', tweet)

  return tweet

In [ ]:
data_clean = [clean_tweet(tweet) for tweet in train_data.text]

In [ ]:
data_labels = train_data.sentiment.values

In [ ]:
set(data_labels)

{0, 4}

In [ ]:
data_labels = train_data.sentiment.values
data_labels[data_labels == 4] = 1

In [ ]:
set(data_labels)

{0, 1}

### **Tokenization**

In [ ]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    data_clean, target_vocab_size = 2**16 # maximum word size in english
)

data_inputs = [tokenizer.encode(sentence) for sentence in data_clean] 

#each inputs are the list of numbers of the sequence of a tweet

In [ ]:
data_inputs

### **Padding**

In [ ]:
# will work in batches in AI
# all our examplem should be of same length
#so we add zeroes at the end to make it of same length
# zer doesnn't use by the tokenizer, it does nbot have any maning so we can use it safely

MAX_LEN = max([len(sentence) for sentence in data_inputs]) #lenth of sentence is the number of words 
data_inputs = tf.keras.preprocessing.siquence.pad_sequences(data_inputs,
                                                            value= 0,
                                                            padding='post', #at the end
                                                            mxlen = MAX_LEN)

### **Spliting into training/testing set**

In [ ]:
test_idx = np.random.randint(0, 800000, 8000)
test_idx = np.concatenate((test_idx, test_idx+800000))

In [ ]:
test_inputs = data_inuts[test_idx]
test_labels = data_labels[test_idx]
train_inputs = np.delete(data_inputs, test_idx, axis =0)
train_labels = np.delete(data_labels, test_idx)